In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['state_labels.csv', 'train_metadata', 'color_labels.csv', 'StateLabels.csv', 'train_images', 'test_metadata', 'PetFinder-BreedLabels.csv', 'BreedLabels.csv', 'train_sentiment', 'ColorLabels.csv', 'test_sentiment', 'PetFinder-ColorLabels.csv', 'PetFinder-StateLabels.csv', 'test', 'train', 'breed_labels.csv', 'test_images']


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from pandas.io.json import json_normalize    
import os
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier

# Thêm dữ liệu

In [3]:
train = pd.read_csv('../input/train/train.csv')
test = pd.read_csv('../input/test/test.csv')


**Xử lý 'Name'**
* **Vấn đề:** Tên vật nuôi có ảnh hưởng đến tốc độ nhận nuôi, những vật nuôi không có tên, vật nuôi tên đặc biệt có các ảnh hưởng khác nhau
* **Cách làm:** Tạo các trường mới thể hiện đặc điểm cần sử dụng có ảnh hưởng đến dự đoán của mô hình (No_name: cho các vật nuôi không có tên, uniq_name: cho các vật nuôi có tên đặc biệt dưới 3 ký tự và duy nhất, Name_length: chiều dài tên, Name_lenght_split: số từ trong tên, ....)
* **Kết quả:** ta thu được các trường mới như bên dưới 
* **Vấn đề nảy sinh:** trong số đó chỉ có No_name và uniq_name có ảnh hưởng tốt tới mô hình, các trường còn lại làm giảm khả năng dự đoán của mô hình, ta sẽ loại bỏ các trường này trong tập train, không sử dụng nữa
* **Bài học:** Những cái tên bình thường không ảnh hưởng đến tốc độ nhận nuôi

In [4]:
#Làm đầy các giá trị null
train['Name'] = train['Name'].fillna('Unnamed')
test['Name'] = test['Name'].fillna('Unnamed')
#Thêm 1 trường binary cho các vật nuôi không có tên
train['No_name'] = 0
train.loc[train['Name'] == 'Unnamed', 'No_name'] = 1
test['No_name'] = 0
test.loc[test['Name'] == 'Unnamed', 'No_name'] = 1
#Thêm 1 trường binary cho các vật nuôi có tên đặc biệt
train['uniq_name'] = 0
noname = train[train['Name'].apply(lambda x: len(str(x))) < 3]['Name'].unique()
train.loc[train['Name'].isin(noname), 'uniq_name'] = 1

test['uniq_name'] = 0
noname = test[test['Name'].apply(lambda x: len(str(x))) < 3]['Name'].unique()
test.loc[test['Name'].isin(noname), 'uniq_name'] = 1
#Độ dài của tên và số từ trong tên
train['Name_length'] = train['Name'].apply(lambda x:len(str(x)))
train['Name_length_split'] = train['Name'].apply(lambda x:len(str(x).split()))
test['Name_length'] = train['Name'].apply(lambda x:len(str(x)))
test['Name_length_split'] = train['Name'].apply(lambda x:len(str(x).split()))
#Những vật nuôi trong tên có chứa kitty
def kitty(x):
    if 'kitty' in x.lower():
        return 1
    else:
        return 0
train['Name_if_kitty'] = train['Name'].apply(lambda x:kitty(str(x)))
test['Name_if_kitty'] = test['Name'].apply(lambda x:kitty(str(x)))
#Những vật nuôi trong tên có chứa puppy
def puppy(x):
    if 'puppy' in x.lower():
        return 1
    else:
        return 0
train['Name_if_puppy'] = train['Name'].apply(lambda x:puppy(str(x)))
test['Name_if_puppy'] = test['Name'].apply(lambda x:puppy(str(x)))

**Xử lý 'Description'**
* **Vấn đề:** Không thể sử dụng rất nhiễn đoạn giới thiệu khác nhau để học nên ta sẽ sử dụng các giá trị hữu ích của nó để trích xuất cho mô hình học
* **Các làm:** Sử dụng độ dài của mô tả và số lượng từ làm đặc tính
* **Kết quả:** Các trường này làm tăng khả năng học của mô hình
* **Bài học:** Có thể biến đổi các giá trị không thể sử dụng được để tạo ra một giá trị mới

In [6]:
train['Description'] = train['Description'].fillna('')
test['Description'] = test['Description'].fillna('')

train['desc_length'] = train['Description'].apply(lambda x: len(x))
train['desc_words'] = train['Description'].apply(lambda x: len(x.split()))

test['desc_length'] = test['Description'].apply(lambda x: len(x))
test['desc_words'] = test['Description'].apply(lambda x: len(x.split()))


**Xử lý 'Age'**
* **Vấn đề:** Trong dữ liệu có một số lượng lớn vật nuôi có độ tuổi trên 12 tháng có số tháng tuổi là bội số của 12 -> giá trị tuổi không thực sự chính xác, có thể quy các vật nuôi khác về cùng 1 dạng
* **Các làm:** (1) Chia hết số tháng cho 12 để lấy số tuổi theo năm (2) Do số vật nuôi có số tháng tuổi dưới 12 tháng lớn nên từ 12 tháng trở lên ta mới quy về bội của 12
* **Kết quả:** Ta có được các trường mới
* **Vấn đề nảy sinh:** Các trường này làm mô hình trở nên tệ hơn


In [7]:
train['AgeYear'] = train['Age']//12
test['AgeYear'] = test['Age']//12

**Xử lý 'Quantyti'**
* **Vấn đề:** Có các trường thông tin đại diện cho nhiều hơn 1 vật nuôi, làm tăng khả năng không được nhận nuôi
* **Các làm:** thêm 1 trường nữa cho các vật nuôi có thông tin riêng biệt
* **Kết quả:** Ta có được các trường mới làm tăng khả năng học của mô hình


In [8]:
train['onlyPet'] = (train['Quantity']==1)
test['onlyPet'] = (test['Quantity']==1)

**Xử lý 'Fee'**
* **Vấn đề:** Số lượng vật nuôi miễn phí chiếm phần lớn dữ liệu
* **Các làm:** thêm 1 trường vật nuôi có miễn phí hay không
* **Kết quả:** Ta có được các trường mới tăng khả năng học của mô hình
* **Bài học:** Khi 1 giá trị chiếm đa số so với các trường còn lại, ta quy nó về biến nhị phân để giảm mất cân bằng


In [9]:
train['Free'] = train['Fee'].apply(lambda x: 1 if x == 0 else 0)
test['Free'] = test['Fee'].apply(lambda x: 1 if x == 0 else 0)

**Xử lý 'Video'**
Tương tự như 'Fee'


In [10]:
train['HaveVideo'] = (train['VideoAmt']!=0)
test['HaveVideo'] = (test['VideoAmt']!=0)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 32 columns):
Type             14993 non-null int64
Name             14993 non-null object
Age              14993 non-null int64
Breed1           14993 non-null int64
Breed2           14993 non-null int64
Gender           14993 non-null int64
Color1           14993 non-null int64
Color2           14993 non-null int64
Color3           14993 non-null int64
MaturitySize     14993 non-null int64
FurLength        14993 non-null int64
Vaccinated       14993 non-null int64
Dewormed         14993 non-null int64
Sterilized       14993 non-null int64
Health           14993 non-null int64
Quantity         14993 non-null int64
Fee              14993 non-null int64
State            14993 non-null int64
RescuerID        14993 non-null object
VideoAmt         14993 non-null int64
Description      14993 non-null object
PetID            14993 non-null object
PhotoAmt         14993 non-null float64
AdoptionSpe

**Lấy giá trị trong file sentiment**
* **Vấn đề:** Trong file sentiment chứa giá trị thông tin đo lường về tỉnh cảm của vật nuôi, cần tận dụng chúng
* **Cách làm:** Đọc input và tách giá trị magnitude và score trong file json
* **Kết quả:** Ta thu được các trường mới có giá trị cho việc học của mô hình

In [12]:
def readFile(fn):
    file = '../input/train_sentiment/'+fn['PetID']+'.json'
    if os.path.exists(file):
        with open(file) as data_file:    
            data = json.load(data_file)  

        df = json_normalize(data)
        mag = df['documentSentiment.magnitude'].values[0]
        score = df['documentSentiment.score'].values[0]
        return pd.Series([mag,score],index=['mag','score']) 
    else:
        return pd.Series([0,0],index=['mag','score'])
    
def readTestFile(fn):
    file = '../input/test_sentiment/'+fn['PetID']+'.json'
    if os.path.exists(file):
        with open(file) as data_file:    
            data = json.load(data_file)  

        df = json_normalize(data)
        mag = df['documentSentiment.magnitude'].values[0]
        score = df['documentSentiment.score'].values[0]
        return pd.Series([mag,score],index=['mag','score']) 
    else:
        return pd.Series([0,0],index=['mag','score'])

In [13]:
train[['SentMagnitude', 'SentScore']] = train[['PetID']].apply(lambda x: readFile(x), axis=1)
test[['SentMagnitude', 'SentScore']] = test[['PetID']].apply(lambda x: readTestFile(x), axis=1)


**Xử lý 'Rescuer'**
* **Vấn đề:** Mã số của các rescuer, với một số rescuer tiêu biểu cứu được rất nhiều vật nuôi, những vật nuôi này cũng có xu hướng được nhận nuôi cao hơn
* **Các làm:** thêm 1 trường là giá trị tổng của mỗi rescuer
* **Kết quả:** Ta có được các trường mới tăng khả năng học của mô hình


In [14]:
df = train.groupby(['RescuerID'])['PetID'].count().reset_index()
df['RescuerIDCount'] = df['PetID']
df = df.drop('PetID', axis=1)
train = pd.merge(train,df, on="RescuerID")

df = test.groupby(['RescuerID'])['PetID'].count().reset_index()
df['RescuerIDCount'] = df['PetID']
df = df.drop('PetID', axis=1)
test = pd.merge(test,df, on="RescuerID")

**Xử lý 'Breed'**
* **Vấn đề:** Số lượng vật nuôi thuần chủng có khả năng được nhận nuôi sớm
* **Các làm:** thêm 1 trường vật nuôi có thuần chủng hay không
* **Kết quả:** Ta có được các trường mới tăng khả năng học của mô hình
* **Bài học:** Ta có thể tạo thêm các trường mới kết hợp thông tin của các trường đã có


In [101]:
train['Pure_breed'] = 0
train.loc[train['Breed2'] == 0, 'Pure_breed'] = 1
test['Pure_breed'] = 0
test.loc[test['Breed2'] == 0, 'Pure_breed'] = 1

**Bỏ các giá trị không chứa thông tin có thể học được**

In [102]:
train_X = train.drop(['Name', 'Description', 'RescuerID', 'PetID', 'AdoptionSpeed', 'VideoAmt', 'AgeYear','Name_length', 'Name_length_split','Name_length_if_num', 'Name_if_kitty', 'Name_if_puppy'], axis=1)
train_y = train['AdoptionSpeed']
test_X = test.drop(['Name', 'Description', 'RescuerID', 'PetID', 'VideoAmt', 'AgeYear', 'Name_length_split','Name_length', 'Name_length_if_num', 'Name_if_kitty', 'Name_if_puppy'], axis=1)


Lấy random các trường để kiểm tra

In [103]:
train_X.sample(5)

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,PhotoAmt,No_name,uniq_name,desc_length,desc_words,onlyPet,Free,HaveVideo,SentMagnitude,SentScore,RescuerIDCount,Pure_breed
2707,1,1,307,0,2,2,5,7,2,2,2,2,2,1,1,0,41326,5.0,0,0,189,31,True,1,False,0.9,0.4,12,1
2572,1,3,307,49,1,1,7,0,2,1,2,2,2,1,1,0,41401,2.0,0,0,183,31,True,1,False,1.5,0.1,111,0
6392,2,1,266,0,1,7,0,0,2,2,2,1,2,1,1,0,41326,7.0,0,0,527,105,True,1,False,1.6,0.0,31,1
8340,2,9,265,0,2,4,7,0,3,1,2,1,1,1,1,0,41336,4.0,0,0,125,25,True,1,False,0.3,0.3,5,1
12584,1,2,307,0,2,5,0,0,2,1,3,3,2,1,1,0,41401,1.0,1,0,20,5,True,1,False,0.0,0.0,1,1


**Mô hình**
* **Cách làm:** Sử dụng mô hình lightGBM để học các đặc trưng, sử dụng KFold để chia dữ liệu thành các phần khác nhau để kiểm chứng chéo, tăng khả năng học đúng của mô hình
* **Kết quả:** Kết quả cao nhất mô hình đạt được là 0.3445 điểm trên tập test của Kaggle

In [151]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from lightgbm import LGBMClassifier

train_meta = np.zeros(train_y.shape)
test_meta = np.zeros(test_X.shape[0])

clf = LGBMClassifier(objective='multiclass', random_state=5)

splits = list(StratifiedKFold(n_splits=4, shuffle=True, random_state=1812).split(train_X, train_y))
for idx, (train_idx, valid_idx) in enumerate(splits):
        X_train = train_X.iloc[train_idx]
        y_train = train_y[train_idx]
        X_val = train_X.iloc[valid_idx]
        y_val = train_y[valid_idx]
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        print(cohen_kappa_score(y_val, y_pred, weights='quadratic'))
        
        y_test = clf.predict(test_X)
        
        train_meta[valid_idx] = y_pred.reshape(-1)
        test_meta += y_test.reshape(-1) / len(splits)

0.37763654614764053
0.36324759467748824
0.38190846015452407
0.3747343127838191


**Xuất pred ra dưới dạng submission theo yêu cầu **

In [111]:
sub = pd.read_csv('../input/test/sample_submission.csv')
sub['AdoptionSpeed'] = test_meta
sub['AdoptionSpeed'] = sub['AdoptionSpeed'].astype(int)
sub.to_csv("submission.csv", index=False)

In [112]:
sub.head(5)

,PetID,AdoptionSpeed
0,e2dfc2935,4
1,f153b465f,3
2,3c90f3f54,1
3,e02abc8a3,4
4,09f0df7d1,4
